# Matrix Operations

_February 20 2024 | Chen Kai Zhang | 014806701_

Observing behaviors of **matrix operations** by applying them to the [2009 Statewide Crime Data](https://www.statsmodels.org/dev/datasets/generated/statecrime.html) dataset.

In [103]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

crime_data = sm.datasets.statecrime
crime_data = crime_data.load_pandas()
crime_data = crime_data.data

crime_data.head()

,violent,murder,hs_grad,poverty,single,white,urban
state,,,,,,,
Alabama,459.9,7.1,82.1,17.5,29.0,70.0,48.65
Alaska,632.6,3.2,91.4,9.0,25.5,68.3,44.46
Arizona,423.2,5.5,84.2,16.5,25.7,80.0,80.07
Arkansas,530.3,6.3,82.4,18.8,26.3,78.4,39.54
California,473.4,5.4,80.6,14.2,27.8,62.7,89.73


In [104]:
def is_square(matrix):
    rows, cols = matrix.shape
    return rows == cols

def is_symmetric(matrix):
    return matrix.equals(matrix.T)

def is_orthogonal(matrix):
    rows, cols = matrix.shape
    if rows == cols:
        identity_matrix = np.eye(rows)
        return np.allclose(matrix @ matrix.T, identity_matrix) # Checks if all elements are equal
    else:
        return False
    
def is_positive_definite(matrix):
    if not is_square(matrix) and not is_symmetric(matrix):
        return False
    eigenvalues = np.linalg.eigvals(matrix)
    return np.all(eigenvalues > 0)

def get_eigens(matrix):
    return np.linalg.eig(matrix)

def compute_inverse(matrix):
    eigenvalues, eigenvectors = get_eigens(matrix)

    Q = eigenvectors
    lmbda_inv = np.diag(1/eigenvalues)
    return Q @ lmbda_inv @ Q.T  # Spectral Decomposition

def compute_square_root(matrix):
    eigenvalues, eigenvectors = get_eigens(matrix)

    Q = eigenvectors
    lmbda_sqrt = np.diag(np.sqrt(eigenvalues))
    return Q @ lmbda_sqrt @ Q.T # Spectral Decomposition

def perform_pca(matrix, k=2):
    # 1. Get the eigens
    eigenvalues, eigenvectors = get_eigens(matrix)

    # 2. Sort by eigenvalues in descending order
    indicies = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[indicies]
    eigenvectors = eigenvectors[:, indicies]

    # 3. Project the data (keep top k components)
    W = eigenvectors[:, :k]
    
    # 4. Return the eigenvalues and eigenvectors
    return eigenvalues[:k], W




Computations for **covariance** and **correlation** matrix will be done one after the other for easier comparison

##### 1. Computing the matrix

In [105]:
crime_data_cov = crime_data.cov()
print(crime_data_cov)

              violent    murder     hs_grad     poverty      single  \
violent  43271.258282  658.3354 -269.241388  234.676188  804.975953   
murder     658.335400   13.2940   -5.640000    6.144800   15.309800   
hs_grad   -269.241388   -5.6400   11.409325   -7.801992   -9.076702   
poverty    234.676188    6.1448   -7.801992    9.675725    7.344569   
single     804.975953   15.3098   -9.076702    7.344569   22.911208   
white    -1626.801965  -28.3890   15.633910   -7.880843  -48.767439   
urban     1851.604044   22.5923  -11.900056  -12.624051   41.928423   

               white        urban  
violent -1626.801965  1851.604044  
murder    -28.389000    22.592300  
hs_grad    15.633910   -11.900056  
poverty    -7.880843   -12.624051  
single    -48.767439    41.928423  
white     183.684596  -126.569354  
urban    -126.569354   432.741194  


In [106]:
crime_data_corr = crime_data.corr()
print(crime_data_corr)

          violent    murder   hs_grad   poverty    single     white     urban
violent  1.000000  0.868000 -0.383188  0.362683  0.808461 -0.577030  0.427892
murder   0.868000  1.000000 -0.457953  0.541799  0.877239 -0.574495  0.297864
hs_grad -0.383188 -0.457953  1.000000 -0.742564 -0.561402  0.341508 -0.169358
poverty  0.362683  0.541799 -0.742564  1.000000  0.493288 -0.186937 -0.195094
single   0.808461  0.877239 -0.561402  0.493288  1.000000 -0.751743  0.421086
white   -0.577030 -0.574495  0.341508 -0.186937 -0.751743  1.000000 -0.448929
urban    0.427892  0.297864 -0.169358 -0.195094  0.421086 -0.448929  1.000000


**Analysis**

Looking at the two matrices side by side, the fact that the correlation matrix standardizes the relationship between $[-1,1]$ makes it a lot easier to understand the relationship strength between two variables. Regardless of my preferences, both clearly show the positive/negative effect a variable has on another variable, but it is only through the correlation matrix I can clearly see whether or not a variable is redundant, which, none of the variables appears to be.

- Just wanted to pointed how although all relationships don't have a correlation high enough to be considered redundant, **murder** and **violent** relationship is at $8.77$, which, is considerably high and makes sense in this dataset


##### 2. Checking if matrix is **square/symmetric/orthogonal**

In [107]:
print(f"IS SQUARE: {is_square(crime_data_cov)}")
print(f"IS SYMMETRIC: {is_symmetric(crime_data_cov)}")
print(f"IS ORTHOGONAL: {is_orthogonal(crime_data_cov)}")

IS SQUARE: True
IS SYMMETRIC: True
IS ORTHOGONAL: False


In [108]:
print(f"IS SQUARE: {is_square(crime_data_corr)}")
print(f"IS SYMMETRIC: {is_symmetric(crime_data_corr)}")
print(f"IS ORTHOGONAL: {is_orthogonal(crime_data_corr)}")

IS SQUARE: True
IS SYMMETRIC: True
IS ORTHOGONAL: False


**Analysis**

Both being the **True** for the first two properties makes a lot of sense since

- **square** : compares variances/relationships between the same set of variables
- **symmetric** : $Cov(x,y) = Cov(y,x)$ and $Corr(x,y) = Corr(y,x)$
    - $x$ and $y$ are two random variables

Determining if a matrix was orthogonal took a bit of math. Since both matrices are square, there is potential that both either could be orthogonal, so to determine if they were, we can use the formula 

$$AA^T = I$$

... where $I$ is the **identity matrix**. With the formula, it showed that neither were orthogonal.


##### 3. Checking if matrix is **positive definite**

In [109]:
print(f"IS_POSITIVE_DEFINITE: {is_positive_definite(crime_data_cov)}")

IS_POSITIVE_DEFINITE: True


In [110]:
print(f"IS_POSITIVE_DEFINITE: {is_positive_definite(crime_data_corr)}")

IS_POSITIVE_DEFINITE: True


**Analysis**

A matrix is **positive definite** if it is :

1. square
2. symmetric
3. all positive $\lambda$ (eigenvalues)

From section 2, we determined that both matrices are square and symmetric so for this scenario, we just have to verify property 3. This is easily achieved with the `np.linalg.eigvals()` method in `numpy`, which will give us all the eigenvalues for a matrix in a list, and verify that all the elements returned are greater than $0$. 

Through this confirmation, it shows that the dataset's variance is robust, no redundant variables, and all lead to a positive stretch.

##### 4. Computing matrix **eigenvalues** and **eigenvectors**

In [111]:
eigenvalues, eigenvectors = get_eigens(crime_data_cov)
print(f"EIGENVALUES:\n{eigenvalues}")
print(f"\nEIGENVECTORS:\n{eigenvectors}")

EIGENVALUES:
[4.34406319e+04 3.67657252e+02 1.12546128e+02 1.67052073e+01
 4.47123980e+00 1.02712177e+00 1.93549041e+00]

EIGENVECTORS:
[[ 0.99803654 -0.0504589  -0.03075742 -0.01482434  0.01392456 -0.00385533
   0.00156037]
 [ 0.0151849  -0.01355376  0.04928396  0.21302951 -0.56450146  0.72068897
  -0.33722507]
 [-0.00621958 -0.00335919 -0.06027943 -0.68426506 -0.48261806 -0.3613952
  -0.4056736 ]
 [ 0.00539354 -0.06188794  0.04947431  0.5799116   0.07048666 -0.43832801
  -0.67845104]
 [ 0.01859539  0.03021564  0.15772355  0.36006835 -0.65784629 -0.39631668
   0.50436757]
 [-0.03769432 -0.22504671 -0.95664658  0.1307852  -0.10216165 -0.02498129
   0.06778347]
 [ 0.0431053   0.97049976 -0.2247095   0.05593343 -0.00754388 -0.01279207
  -0.04928177]]


In [112]:
eigenvalues, eigenvectors = get_eigens(crime_data_corr)
print(f"EIGENVALUES:\n{eigenvalues}")
print(f"\nEIGENVECTORS:\n{eigenvectors}")

EIGENVALUES:
[4.06197705 1.45610196 0.66628181 0.47235681 0.06505327 0.15627176
 0.12195733]

EIGENVECTORS:
[[ 0.43087353  0.14760338  0.34635648 -0.35595713  0.22206973 -0.55614757
   0.43275613]
 [ 0.45140908 -0.01438198  0.38921085 -0.21847911 -0.68097708  0.24818803
  -0.26735316]
 [-0.33382774  0.40532197  0.6271116   0.02076648  0.21043968  0.4853986
   0.22511391]
 [ 0.29655969 -0.62570407 -0.05851202 -0.0431628   0.18488676  0.50152191
   0.47911664]
 [ 0.47364909  0.0617134   0.09420384  0.11893395  0.61543067  0.15359427
  -0.58862742]
 [-0.37022653 -0.28293657  0.0444035  -0.81333025  0.17241444  0.022798
  -0.29858829]
 [ 0.22311005  0.58168142 -0.56658049 -0.38420795 -0.0023749   0.34330105
   0.1593053 ]]


**Analysis**

The eigenvalues and the eigenvectors for the **covariance matrix** tells us that rate of violent crime is the dominating factor in variance for the dataset. Since the values are not standardized, it's hard to interpret more information from that. However, the **correlation matrix** is a lot more interesting and tells us a lot more hidden details about the data.

_Exploring **eigenvalues**_ :

PC1's eigenvalue is absurdly high, indicating its importance on capturing the overall trend of the dataset.

_Exploring **eigenvectors**_ :

- Highest positive value for `single` (0.4736) indicate that an increase in number of single-parent household often contribute to increased crime rates 
- Large positive values for `violent` (0.4514) and `murder` (0.4309) suggests high single-parent households lead to higher violent crimes leads to higher murder rates
- Large negative value for `white` (-0.3702) and `hs_grad` (-0.3338) show that regions with high white population and high school graduates lead to lower crime rates



##### 5. Computing matrix **inverse** and **square root** using **spectral decomposition**

In [113]:
print(f"INVERSE:\n{compute_inverse(crime_data_cov)}")
print(f"\nSQUARE ROOT:\n{compute_square_root(crime_data_cov)}")

INVERSE:
[[ 1.10509235e-04 -4.93529269e-03  1.50476531e-04  7.98306465e-04
  -5.20849393e-04  5.65706658e-06 -1.35639473e-04]
 [-4.93529269e-03  6.38441184e-01 -1.30715514e-01 -1.90828467e-01
  -2.78242376e-01 -1.51831608e-02  1.14235502e-03]
 [ 1.50476531e-04 -1.30715514e-01  2.92339445e-01  2.65040246e-01
   8.99040945e-02  7.66972157e-04  1.34648995e-02]
 [ 7.98306465e-04 -1.90828467e-01  2.65040246e-01  4.46151444e-01
  -5.47406573e-03 -1.05524510e-02  2.42945228e-02]
 [-5.20849393e-04 -2.78242376e-01  8.99040945e-02 -5.47406573e-03
   3.89124515e-01  4.37933916e-02 -5.82605727e-03]
 [ 5.65706658e-06 -1.51831608e-02  7.66972157e-04 -1.05524510e-02
   4.37933916e-02  1.46089453e-02  5.11430698e-04]
 [-1.35639473e-04  1.14235502e-03  1.34648995e-02  2.42945228e-02
  -5.82605727e-03  5.11430698e-04  4.62465647e-03]]

SQUARE ROOT:
[[ 2.07666683e+02  3.12264092e+00 -1.24306538e+00  1.13285015e+00
   3.74887735e+00 -7.32177992e+00  8.09722394e+00]
 [ 3.12264092e+00  1.62125212e+00 -1.436

In [114]:
print(f"INVERSE:\n{compute_inverse(crime_data_corr)}")
print(f"\nSQUARE ROOT:\n{compute_square_root(crime_data_corr)}")

INVERSE:
[[ 4.78187364 -3.74317829  0.10573007  0.51654859 -0.51860407  0.01594878
  -0.58694847]
 [-3.74317829  8.48743711 -1.60984831 -2.16427662 -4.85596003 -0.75028523
   0.08664491]
 [ 0.10573007 -1.60984831  3.33539588  2.78473312  1.45356162  0.03511122
   0.94612217]
 [ 0.51654859 -2.16427662  2.78473312  4.3168389  -0.08150341 -0.44486832
   1.57204242]
 [-0.51860407 -4.85596003  1.45356162 -0.08150341  8.91531265  2.84098501
  -0.58011335]
 [ 0.01594878 -0.75028523  0.03511122 -0.44486832  2.84098501  2.68343821
   0.14419071]
 [-0.58694847  0.08664491  0.94612217  1.57204242 -0.58011335  0.14419071
   2.00127936]]

SQUARE ROOT:
[[ 0.78571279  0.41938692 -0.10626304  0.1127234   0.33197574 -0.20074077
   0.17962982]
 [ 0.41938692  0.7349812  -0.12457755  0.24091615  0.40505268 -0.19551388
   0.08979827]
 [-0.10626304 -0.12457755  0.86628336 -0.39230258 -0.22234209  0.11198412
  -0.08284917]
 [ 0.1127234   0.24091615 -0.39230258  0.84206832  0.18945969 -0.02295971
  -0.1727725

**Analysis**

The inverses showed behavior that I personally found really interesting and against the norm of what we are told. The inverse correlation matrix does make more sense to me as it shows that murder rates and single-parent households are inverted, however both show that increase in high school graduate have some positive affect on the increase of poverty, which is very much against what we know. 

The square root matrices made a lot more sense to me since it showed that :

- Urban areas experience more violent crimes
- Increased high school graduates lead to lower poverty 
- Large number of single-parent households lead to more murder
- Increase in violent crimes suggest low white population (not to offend anyone but its an ongoing issue in America)

_Full Analysis_

---

_Exploring **inverse covariance matrix**_ : 

- Diagonals :
    - High values for `murder` (0.638), `poverty` (0.446), and `single` (0.389) indicate that these three features they have low conditional variance
    - Low values for `violent` (0.0001) and `white` (0.0015) indicate high conditional variance
- Off-Diagonals :
    - `murder` has strong negative relationships with `violent` (-0.0049), `single` (-0.2782), and `poverty` (-0.1908), suggesting that higher murder rates associate to low values in those other features
    - `hs_grad` and `poverty have the highest positive value (0.265), indicating that higher graduation rates relate to higher poverty
        - This I find very counterintuitive and interesting, but may just be this dataset
    - `single` and `white` (0.0438) show a positive conditional relationship

_Exploring **square root covariance matrix**_ :

- Diagonals : 
    - High values for `violent` (207.67), `white` (11.08), and `urban` (18.999), suggesting that these features dominate the variance
- Off-Diagonals : 
    - `violent` is a major player in determining the variance out of the 3 above
    - `violent` and `white` (-7.322) suggests that as violent crime goes up, the `white` population goes down
    - `violent` and `urban` (8.097) suggests that as violent crimes goes up, the `urbanized` areas increase

_Exploring **inverse correlation matrix**_ :

- Diagonal :
    - High values for `single` (8.915) and `murder` (8.487) indicate that they are conditionally independent
    - Low values for `urban` (2.001) shows how it doesn't really play a factor much in influencing other variables
- Off-Diagonal : 
    - `murder` and `single` (-4.856) have a strong negative partial correlation
    - `hs_grad` and `poverty` (2.785) & `single` and `white` (2.841) have a strong positive correlation

_Exploring **square root correlation matrix**_ :

- Off-Diagonal :
    - `hs_grad` and `poverty` (-0.392) indicate a negative relationship in the structure.
    - `murder` and `single` (0.405) show a positive contribution to correlation.


##### 6. Performing **PCA**

In [115]:
eigenvalues, eigenvectors = perform_pca(crime_data_cov)
print(f"PCA RESULTS:")
print(f"\nEIGENVALUES:\n{eigenvalues}")
print(f"\nEIGENVECTORS:\n{eigenvectors}")

PCA RESULTS:

EIGENVALUES:
[43440.63189141   367.65725234]

EIGENVECTORS:
[[ 0.99803654 -0.0504589 ]
 [ 0.0151849  -0.01355376]
 [-0.00621958 -0.00335919]
 [ 0.00539354 -0.06188794]
 [ 0.01859539  0.03021564]
 [-0.03769432 -0.22504671]
 [ 0.0431053   0.97049976]]


In [116]:
eigenvalues, eigenvectors = perform_pca(crime_data_corr)
print(f"PCA RESULTS:")
print(f"\nEIGENVALUES:\n{eigenvalues}")
print(f"\nEIGENVECTORS:\n{eigenvectors}")

PCA RESULTS:

EIGENVALUES:
[4.06197705 1.45610196]

EIGENVECTORS:
[[ 0.43087353  0.14760338]
 [ 0.45140908 -0.01438198]
 [-0.33382774  0.40532197]
 [ 0.29655969 -0.62570407]
 [ 0.47364909  0.0617134 ]
 [-0.37022653 -0.28293657]
 [ 0.22311005  0.58168142]]


**Analysis**

From the PCA results, it really shows the importance in standardizing the data. Although both covariance and correlation matrix identifies `violent` and `urban` as the top influence on data variance, it is through the latter that we are able to observe how other features influence the data variance.

In covariance matrix PCA, we realize how overshadowing the raw scale of `violent` can be on other features. PC1's near 98% influence on the variance alongside `violent`'s high influence in that component, we could easily be mislead to believe that is the only valuable feature. It is through the correlation matrix PCA that we understand that not only is PC1 is not the only influential component, but that `single` has the most influence on the component, not `violent`. In addition, we are also able to recognize the significance of PC2 and how demographic and family structures (`urban` and `single`) play a role in data variance. 

_Exploring **covariance matrix PCA**_ :

- PC1 (43440.63) captures the majority of the variance while PC2 (367.66)
- PC1 is largely influenced by `violent` (0.9989)
- PC2 is mostly influenced by `urban` (0.9705) with a some contributions from `white` (-0.225)

_Exploring **correlation matrix PCA**_ :

- PC1 (4.062) and PC2 (1.456) explains most of the variance by capturing $\frac{4.062}{7} + \frac{1.456}{7} * 100 = 78.9%$ of the variance in the dataset
- PC1 is largely influenced by `single` (0.4736), `murder` (0.4514), and `violent` (0.4309)
- PC2 is mostly influenced by `single` (-0.626), `urban` (0.582), and `hs_grad` (0.405)